In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm

In [7]:
model = YOLO('../model/yolov8m-face.pt')
cap = cv2.VideoCapture('testvideoShorts.mp4')
fps = round(cap.get(cv2.CAP_PROP_FPS))
w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
saveVideo = cv2.VideoWriter('outputShort.mp4', fourcc, fps, (w, h))

for i in tqdm(range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))):
# while(True):
    retval, frame = cap.read()
    
    if not(retval):	# 프레임정보를 정상적으로 읽지 못하면
        break  # while문을 빠져나가기
    
    else:
        results = model(frame, stream=True ,verbose=False)
        
        for result in results:
            for box in result.boxes:
                left_x = int(box.xyxy.tolist()[0][0])
                left_y = int(box.xyxy.tolist()[0][1])
                right_x = int(box.xyxy.tolist()[0][2])
                right_y = int(box.xyxy.tolist()[0][3])
                
                face = frame[left_y:right_y, left_x:right_x]
                
                frame[left_y:right_y, left_x:right_x] = cv2.blur(face, (40, 40))
				
        saveVideo.write(frame)
            
        if cv2.waitKey(1) == ord('q'):
            break

    
if cap.isOpened():	# 영상 파일(카메라)이 정상적으로 열렸는지(초기화되었는지) 여부
    cap.release()	# 영상 파일(카메라) 사용을 종료
    saveVideo.release()
    
cv2.destroyAllWindows() 
cv2.waitKey(1) #맥에서 창 종료가 안되는 문제 해결
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 1674/1674 [07:49<00:00,  3.57it/s]


-1

In [ ]:
# model = YOLO('../model/yolov8m-face.pt')
# cap = cv2.VideoCapture('testvideoShorts.mp4')
# fps = round(cap.get(cv2.CAP_PROP_FPS))
# w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# saveVideo = cv2.VideoWriter('outputShort.mp4', fourcc, fps, (w, h))

# for i in tqdm(range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))):
# # while(True):
#     retval, frame = cap.read()
    
#     if not(retval):	# 프레임정보를 정상적으로 읽지 못하면
#         break  # while문을 빠져나가기
    
#     else:
#         results = model(frame, verbose=False)
#         boxes = results[0].boxes
        
#         for box in boxes:
#             left_x = int(box.xyxy.tolist()[0][0])
#             left_y = int(box.xyxy.tolist()[0][1])
#             right_x = int(box.xyxy.tolist()[0][2])
#             right_y = int(box.xyxy.tolist()[0][3])
            
#             face = frame[left_y:right_y, left_x:right_x]
            
#             frame[left_y:right_y, left_x:right_x] = cv2.blur(face, (40, 40))
            
#         saveVideo.write(frame)
            
#         if cv2.waitKey(1) == ord('q'):
#             break

    
# if cap.isOpened():	# 영상 파일(카메라)이 정상적으로 열렸는지(초기화되었는지) 여부
#     cap.release()	# 영상 파일(카메라) 사용을 종료
#     saveVideo.release()
    
# cv2.destroyAllWindows() 
# cv2.waitKey(1) #맥에서 창 종료가 안되는 문제 해결
# cv2.waitKey(1)
# cv2.waitKey(1)
# cv2.waitKey(1)